In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import avg, count, expr
from dotenv import load_dotenv
import os

# Carregar variáveis de ambiente do caminho dentro do container
load_dotenv('../.env_kafka_connect')

pg_host = os.getenv("POSTGRES_HOST")
pg_port = os.getenv("POSTGRES_PORT", "5432")
pg_db = os.getenv("POSTGRES_DB")
pg_user = os.getenv("POSTGRES_USER")
pg_password = os.getenv("POSTGRES_PASSWORD")

# JDBC URL e propriedades de conexão
#jdbc_url = f"jdbc:postgresql://{pg_host}:{pg_port}/{pg_db}"
jdbc_url = f"jdbc:postgresql://host.docker.internal:5432/postgres"
jdbc_properties = {
    "user": pg_user,
    "password": pg_password,
    "driver": "org.postgresql.Driver"
}

# Criar SparkSession
spark = SparkSession.builder \
    .appName("Pipeline - Gold") \
    .config("spark.jars", "../jars/postgresql-42.6.2.jar") \
    .getOrCreate()

# Ler as tabelas Silver do PostgreSQL
df_pre = spark.read.jdbc(url=jdbc_url, table="public.dadostesouropre_silver", properties=jdbc_properties)
df_ipca = spark.read.jdbc(url=jdbc_url, table="public.dadostesouroipca_silver", properties=jdbc_properties)

# Registrar views temporárias para SQL
df_ipca.createOrReplaceTempView("ipca_silver")
df_pre.createOrReplaceTempView("pre_silver")

# Consulta agregada para IPCA
df_ipca_gold = spark.sql("""
SELECT
  Data_Base,
  Tipo,
  COUNT(*) AS qtde_registros,
  AVG(CompraManha) AS compra_manha_media,
  AVG(VendaManha) AS venda_manha_media,
  AVG(PUCompraManha) AS pu_compra_manha_media,
  AVG(PUVendaManha) AS pu_venda_manha_media,
  AVG(PUBaseManha) AS pu_base_manha_media,
  AVG(100 * (VendaManha - CompraManha) / CompraManha) AS percentual_diferenca_compra_venda
FROM ipca_silver
GROUP BY Data_Base, Tipo
ORDER BY Data_Base, Tipo
""")

# Consulta agregada para Pré
df_pre_gold = spark.sql("""
SELECT
  Data_Base,
  Tipo,
  COUNT(*) AS qtde_registros,
  AVG(CompraManha) AS compra_manha_media,
  AVG(VendaManha) AS venda_manha_media,
  AVG(PUCompraManha) AS pu_compra_manha_media,
  AVG(PUVendaManha) AS pu_venda_manha_media,
  AVG(PUBaseManha) AS pu_base_manha_media,
  AVG(100 * (VendaManha - CompraManha) / CompraManha) AS percentual_diferenca_compra_venda
FROM pre_silver
GROUP BY Data_Base, Tipo
ORDER BY Data_Base, Tipo
""")

# Gravar as tabelas Gold no PostgreSQL
df_ipca_gold.write.jdbc(url=jdbc_url, table="public.dadostesouroipca_gold", mode="overwrite", properties=jdbc_properties)
df_pre_gold.write.jdbc(url=jdbc_url, table="public.dadostesouropre_gold", mode="overwrite", properties=jdbc_properties)

print("Pipeline Gold executado com sucesso!")

Pipeline Gold executado com sucesso!
